sempre que adicionar ou mudar uma dependencia, usar pip freeze > requirements.txt( usar dentro da venv)


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import requests
import re
import pyarrow.parquet as pq

In [ ]:
dadosAcumulados = [] # lista para armazenar os dados acumulados

page = 1 # variavel para controlar a paginação
i = 1

while True: 

    url = f'https://books.toscrape.com/catalogue/page-{page}.html'

    # resposta com o conteudo da pagina
    response = requests.get(url) 
    
    # verifica se a resposta é diferente de 200, se for, sai do loop
    if response.status_code != 200: 
        break

    result = response.text 

    soup = BeautifulSoup(result, 'lxml')
    
    # busca diretamenta todas as tags li que estão dentro de uma tag ol com a classe row
    for item in soup.select('ol.row li'): 
        

        # busca tags a que tenham o atributo title
        title_aux = item.find("a", attrs={"title":True}) 
        # extrair o texto que está dentro do atributo title de cada tag a encontrada 
        title = title_aux.get('title') 

        # busca tags p que tenham a classe star-rating
        star_aux = item.find("p", class_="star-rating")
        # extrair o texto que está dentro do atributo class na posicao 1 de cada tag p encontrada(é a informação que tem a avaliação do livro) 
        star = star_aux.get('class')[1]

        # busca tags p que tenham a classe price_color
        price_aux = item.find("p", class_="price_color")
        # extrair o texto que está dentro de cada tag p encontrada 
        price = price_aux.getText()

        # busca tags p que tenham a classe instock availability
        availability_aux = item.find("p", class_="instock availability")
        # extrair o texto que está dentro de cada tag p encontrada
        availability = availability_aux.getText()

        # Passa todas as informações encontradas como um dicionario para a varivavel dadosAcumulados, contruindo uma lista de dicionarios
        dadosAcumulados.append({'title':title, 'rating':star, 'price':price, 'availability':availability})

        i = i + 1
        print("loop for: ",i)

    page = page + 1 

    print("pagina: ",page)


In [ ]:
# Para cada item dentro da lista dadosAcumulados, crie um novo dicionário com apenas duas informações
starTable = [{"title": item["title"], "stars": item["rating"]} for item in dadosAcumulados]
priceTable = [{"title": item["title"], "price": item["price"]} for item in dadosAcumulados]
availabilityTable = [{"title": item["title"], "disponibility": item["availability"]} for item in dadosAcumulados]

In [ ]:
# criar um dataframe com os dados acumulados
starDF = pd.DataFrame(starTable)
priceDF = pd.DataFrame(priceTable)
availabilityDF = pd.DataFrame(availabilityTable)

In [41]:
# starDF.to_parquet("BD/BRONZE/starDF.parquet", engine="pyarrow", compression="snappy", partition_cols=["stars"])
starDF.to_parquet("BD/BRONZE/starDF.parquet", engine="pyarrow", compression="snappy")
priceDF.to_parquet("BD/BRONZE/priceDF.parquet", engine="pyarrow", compression="snappy")
availabilityDF.to_parquet("BD/BRONZE/availabilityDF.parquet", engine="pyarrow", compression="snappy")

In [43]:
# Lê o arquivo parquet
table = pq.read_table('BD/BRONZE/starDF.parquet')

# Acessar metadados
metadata = table.schema.metadata
print(metadata)  # Mostra os metadados atuais

# Adicionar/alterar metadados
new_metadata = {b'table': b'star_table',
                b'description': b'table containing the titles and ratings of extracted books', 
                b'columns': b'title: string, stars: string',
                b'created': b'26/05/2025',
                b'data origin': b'https://books.toscrape.com/catalogue/category/books_1/page-1.html',}
table = table.replace_schema_metadata(new_metadata)

# Chama o modulo do pyarrow parquet para sobreescrever o arquivo com os novos metadados
pq.write_table(table, 'BD/BRONZE/starDF.parquet')

{b'table': b'star_table', b'description': b'table containing the titles and ratings of extracted books', b'columns': b'title: string, stars: string', b'created': b'26/05/2025', b'data origin': b'https://books.toscrape.com/catalogue/category/books_1/page-1.html'}


In [45]:
# Lê o arquivo parquet
table = pq.read_table('BD/BRONZE/priceDF.parquet')

# Acessar metadados
metadata = table.schema.metadata
print(metadata)  # Mostra os metadados atuais

# Adicionar/alterar metadados
new_metadata = {b'table': b'price_table',
                b'description': b'table containing the titles and prices of extracted books', 
                b'columns': b'title: string, price: string',
                b'created': b'26/05/2025',
                b'data origin': b'https://books.toscrape.com/catalogue/category/books_1/page-1.html',}
table = table.replace_schema_metadata(new_metadata)

# Chama o modulo do pyarrow parquet para sobreescrever o arquivo com os novos metadados
pq.write_table(table, 'BD/BRONZE/priceDF.parquet')

{b'table': b'price_table', b'description': b'table containing the titles and prices of extracted books', b'columns': b'title: string, price: string', b'created': b'26/05/2025', b'data origin': b'https://books.toscrape.com/catalogue/category/books_1/page-1.html'}


In [47]:
# Lê o arquivo parquet
table = pq.read_table('BD/BRONZE/availabilityDF.parquet')

# Acessar metadados
metadata = table.schema.metadata
print(metadata)  # Mostra os metadados atuais

# Adicionar/alterar metadados
new_metadata = {b'table': b'availability_table',
                b'description': b'table containing the titles and availability of extracted books', 
                b'columns': b'title: string, disponibility: string',
                b'created': b'26/05/2025',
                b'data origin': b'https://books.toscrape.com/catalogue/category/books_1/page-1.html',}
table = table.replace_schema_metadata(new_metadata)

# Chama o modulo do pyarrow parquet para sobreescrever o arquivo com os novos metadados
pq.write_table(table, 'BD/BRONZE/availabilityDF.parquet')

{b'table': b'availability_table', b'description': b'table containing the titles and availability of extracted books', b'columns': b'title: string, disponibility: string', b'created': b'26/05/2025', b'data origin': b'https://books.toscrape.com/catalogue/category/books_1/page-1.html'}


In [ ]:
# Inicializar a SparkSession

spark = SparkSession.builder \
    .appName("DataPipe") \
    .master("local[*]") \
    .getOrCreate()

df = spark.read.parquet("BD/BRONZE/availabilityDF.parquet")

df.show(truncate=False)


spark.stop()



+----------------------------------------------------------------------------------------------+----------------------------------+
|title                                                                                         |disponibility                     |
+----------------------------------------------------------------------------------------------+----------------------------------+
|A Light in the Attic                                                                          |\n\n    \n        In stock\n    \n|
|Tipping the Velvet                                                                            |\n\n    \n        In stock\n    \n|
|Soumission                                                                                    |\n\n    \n        In stock\n    \n|
|Sharp Objects                                                                                 |\n\n    \n        In stock\n    \n|
|Sapiens: A Brief History of Humankind                                      

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Criar SparkSession
spark = SparkSession.builder.appName("EmptyDF").getOrCreate()

# 1. Criar DataFrame vazio com schema definido
schema = StructType([])  # Schema vazio
df = spark.createDataFrame([], schema)

# 2. Adicionar colunas uma a uma
from pyspark.sql.functions import lit

# Adicionar coluna de strings
df = df.withColumn("nome", lit(None).cast(StringType()))

# Adicionar coluna de inteiros
df = df.withColumn("idade", lit(None).cast(IntegerType()))

# Adicionar coluna de doubles com valor padrão 0.0
df = df.withColumn("salario", lit(0.0).cast(DoubleType()))

df.printSchema()
df.show()